In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Mounting

In [54]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Training

In [2]:
from keras.models import Sequential
# from tensorflow.keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers.core import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from keras.losses import CategoricalCrossentropy

In [4]:
model = Sequential()
model.add(Convolution2D(64, kernel_size=(3, 3), strides=(1, 1),padding="same",
                input_shape=(64,64,3),  activation='relu', data_format='channels_last'
                 ))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Dropout(0.25))
model.add(Dense(units=49,activation='softmax'))

model.compile(loss=CategoricalCrossentropy(),optimizer='rmsprop',metrics=['accuracy'])

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.5,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(r'/content/gdrive/MyDrive/datasets/train',
                                                  target_size = (64, 64),
                                                  batch_size = 256,
                                                  class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(r'/content/gdrive/MyDrive/datasets/test',
                                              target_size = (64, 64),
                                                batch_size = 256,
                                                class_mode = 'categorical')

FileNotFoundError: ignored

In [70]:
if(training_set):
  print("Training set available", training_set)

if(test_set):
  print("Test set available", test_set)

Training set available <keras.preprocessing.image.DirectoryIterator object at 0x7fcf2b67ca60>
Test set available <keras.preprocessing.image.DirectoryIterator object at 0x7fcf2b67c8e0>


In [73]:
history = model.fit(training_set, validation_data = test_set, epochs=1)

AttributeError: ignored

In [ ]:
model.summary()
model.save("model.h5")
model.save_weights("weights.h5")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 64, 64, 64)        1792      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 31, 31, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 29, 29, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 3136)              0         
                                                                 
 dense_4 (Dense)             (None, 1000)             

# Prediction

In [6]:
from keras.models import load_model
import cv2
import numpy as np


model = load_model("/content/drive/MyDrive/model.h5")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 1000)              3

In [7]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

In [ ]:
import cv2
import gradio as gr
import numpy as np

classes = {
    3 : 'bhudha temple',
    1 : 'Badrinath temple',
    16 : 'india gate',
    31 : 'charminar',
    48 : 'hindu temple',
    66 : "statue of unity"

}


def predict_image(img):
  # test = cv2.imread("https://cdn.britannica.com/38/189838-050-83C7395E/India-War-Memorial-arch-New-Delhi-Sir.jpg")
  # test = cv2.resize(test, (64,64))
  # test = np.reshape(test, [1,64,64,3])
  img = img.reshape((-1, 64, 64, 3))
  predict_x = model.predict(img)

  class_ = int(np.argmax(predict_x, axis=1))
  print(classes.get(class_))
  print( "------", class_)
  # prediction=class_

  # print(prediction)
  # return {classes[i]: float(prediction[i]) for i in range(5)}
  return classes.get(class_)

# image = gr.inputs.Image(shape=(64,64))

# label = gr.outputs.Label(num_top_classes=5)

gr.Interface(fn=predict_image, inputs=gr.Image(shape=(64, 64)),
             outputs=gr.Label(num_top_classes=6)).launch(debug='True', share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://72eecbaed65f441321.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
